In [5]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [6]:
# URL of the Amazon search results page
url = 'https://www.flipkart.com/search?q=lenovo%20legion&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off'

In [7]:
def switch_tab(source,tab_index):
    link = source.get('href')
    driver.execute_script(f'window.open("{link}", "_blank");')

    # Wait for the new window to open and then switch to it
    WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > tab_index)
    driver.switch_to.window(driver.window_handles[tab_index])

    # Wait for the new tab to load its content
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'fonts-loaded'))
    )

    # Get the HTML content of the new tab
    new_tab_content = driver.page_source
    soup = BeautifulSoup(new_tab_content, 'html.parser')

    return soup

In [16]:
# Set up the Selenium WebDriver
driver = webdriver.Chrome()
driver.maximize_window()

# Load the page
driver.get(url)

# Wait for the page to fully load
time.sleep(5)

# Get the HTML content of the page
html_content = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find elements containing product information
products = soup.find_all('a', rel='noopener noreferrer', limit=10)

data = []

for product in products:
    product_details = {}
    name = product.find('div', class_='_4rR01T')
    name = name.text if name else "No name available"

    # Extract the price of the product
    price = product.find('div', class_='_30jeq3 _1_WHN1')
    price = price.text if price else "No price available"

    # Extract the rating of the product
    rating = product.find('div', class_='_3LWZlK')
    rating = rating.text if rating else "No rating available"

    num_ratings_reviews_container = product.find('span', class_='_2_R_DZ')
    if num_ratings_reviews_container:
        ratings_reviews_text = num_ratings_reviews_container.get_text().strip()
        
        # Split the text to isolate ratings and reviews
        ratings_text, reviews_text = ratings_reviews_text.split('&')
        
        # Extract just the numbers
        num_ratings = ''.join(filter(str.isdigit, ratings_text))
        num_reviews = ''.join(filter(str.isdigit, reviews_text))
    else:
        num_ratings = "No ratings"
        num_reviews = "No reviews"

    # print(f"Name: {name}")
    # print(f"Price: {price}")
    # print(f"Rating: {rating}")
    # print(f"Number of Ratings: {num_ratings}")
    # print(f"Number of Reviews: {num_reviews}")
    # print('-' * 50)
    
    product_details['Product Name'] = name
    product_details['Price'] = price
    product_details['Rating'] = rating
    product_details['Number of Ratings'] = num_ratings
    product_details['Number of Reviews'] = num_reviews
    if num_reviews != "No reviews" and num_reviews != "0":
        soup = switch_tab(product,1)

        # Find the element with the reviews or link to reviews
        div_element = soup.find('div', class_='col JOpGWq')

        if div_element:
            # Find all <a> tags within the div
            all_a_tags = div_element.find_all('a')
            # Check if there are any <a> tags
            if len(all_a_tags) > 10:
                # Get the last <a> tag
                last_a_tag = all_a_tags[-1]

                soup = switch_tab(last_a_tag,2)

                reviews_container = soup.find_all('div',class_='col _2wzgFH K0kLPL')
                review_summary_list = []
                review_details_list = []
                for review in reviews_container:
                    review_summary = review.find('p',class_='_2-N8zT').text if review.find('p',class_='_2-N8zT') else "No summary"
                    review_details = review.find('div',class_='t-ZTKy').text if review.find('div',class_='t-ZTKy') else "No details"
                    review_summary_list.append(review_summary)
                    review_details_list.append(review_details)
                    # print(f"{reviews_container.index(review)}. review_summary: {review_summary}")
                    # print(f"{reviews_container.index(review)}. review_details: {review_details}")
                product_details['Review Summary'] = review_summary_list
                product_details['Review Details'] = review_details_list
                driver.close()
            else:
                review_summary_list = []
                review_details_list = []
                reviews_summary_container = soup.find_all('p',class_='_2-N8zT')
                for reviews_summary in reviews_summary_container:
                    reviews_summary = reviews_summary.text
                    review_summary_list.append(reviews_summary)
                #     # print(f"{reviews_summary_container.index(reviews_summary)}. review_summary: {review_summary}")
                reviews_details_container = soup.find_all('div',class_='t-ZTKy')
                for review_details in reviews_details_container:
                    # print(reviews_details_container.index(review_details))
                    review_details = review_details.text
                    review_details_list.append(review_details)
                    # print(f". review_details: {review_details}")
                product_details['Review Summary'] = review_summary_list
                product_details['Review Details'] = review_details_list

            # print('-' *50)
            
            driver.switch_to.window(driver.window_handles[1])
            driver.close()
        driver.switch_to.window(driver.window_handles[0])
    data.append(product_details)

# Close the browser
driver.quit()

In [17]:
data

[{'Product Name': 'Lenovo IdeaPad Gaming 3 AMD Ryzen 5 Hexa Core 7535HS - (16 GB/512 GB SSD/Windows 11 Home/4 GB Graphics...',
  'Price': '₹56,990',
  'Rating': '4.3',
  'Number of Ratings': '68',
  'Number of Reviews': '8',
  'Review Summary': ['Classy product',
   'Worth the money',
   'Horrible',
   'Best in the market!',
   'Super!',
   'Simply awesome',
   'Simply awesome',
   'Excellent'],
  'Review Details': ['First Impression: Great build no wobble, full fledged 70 watt RTX2050 gives nearly 3050 75w level performance. 16gb DDR5 ram. 7535HS is good. Only cons are: SSD maxes out at 3500/2000mb/s, though upgradation to a gen4 2280 SSD is possible. And no Ms office. Will update more with timeREAD MORE',
   'Overall the product is really nice. Lenovo has included some tweaking features. The only problem that I felt is with the hinge. The laptop screen wobble sometimes.READ MORE',
   'Lenovo worst company I purchased this laptop and from 2nd day trackpad started auto clicking and lag

In [18]:
df = pd.DataFrame(data)
df

,Product Name,Price,Rating,Number of Ratings,Number of Reviews,Review Summary,Review Details
0,Lenovo IdeaPad Gaming 3 AMD Ryzen 5 Hexa Core ...,"₹56,990",4.3,68,8,"[Classy product, Worth the money, Horrible, Be...","[First Impression: Great build no wobble, full..."
1,Lenovo IdeaPad Gaming 3 AMD Ryzen 5 Hexa Core ...,"₹47,990",3.7,7,1,"[Battery issue, rest is good]",[product is good with this price tag; RTX gpu ...
2,Lenovo Legion 5 AMD Ryzen 7 Octa Core 5800H - ...,"₹99,990",4.6,7,0,NaN,NaN
3,Lenovo Legion 5 Pro AMD Ryzen 7 Octa Core 6800...,"₹1,24,990",4.3,25,1,[Good choice],[Awesome laptop with a few niggles. The displa...
4,Lenovo Legion 5 Pro Intel Core i7 12th Gen 127...,"₹1,44,990",No rating available,No ratings,No reviews,NaN,NaN
5,Lenovo Legion 5 Pro AMD Ryzen 7 Octa Core 5800...,"₹1,19,990",4.7,3,0,NaN,NaN
6,Lenovo Legion 5 AMD AMD Ryzen 7 Octa Core 5800...,"₹98,990",4.4,88,18,"[Good quality product, Highly recommended, Wor...","[Very powerful gaming machine, but I highly re..."
7,Lenovo Legion 5 Pro AMD Ryzen 7 Octa Core 5800...,"₹1,04,990",4.5,56,7,"[Absolute rubbish!, Mind-blowing purchase, Won...",[1 star rating is for Flipkart's smart upgrade...
8,Lenovo Legion Slim 5 AMD Ryzen 7 Octa Core 784...,"₹1,37,490",No rating available,No ratings,No reviews,NaN,NaN
9,Lenovo Legion 5 Pro AMD Ryzen 7 Octa Core 6800...,"₹1,09,990",4.1,25,3,"[Wonderful, Highly recommended, Must buy!]",[It exceeded my expectations. Its powerful per...


In [19]:
df.to_csv('D:\Customer Review Analysis Project\scraped_data.csv', index=False)